<a href="https://colab.research.google.com/github/MD-Ryhan/NLP-Preprocesing/blob/main/NLP_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download Dependencies 

In [2]:
pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=6809b91eafe5763d1b3ff3f03e666c3cd9493ce4b388a3c886855f7bc4f24bea
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [3]:
pip install emoticon-fix

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for emoticon-fix: filename=emoticon_fix-0.0.2-py3-none-any.whl size=3444 sha256=a282a933810cefca25800b17272c9d9f9de16c1e8d77ad9197efbc290bbc1aa1
  Stored in directory: /root/.cache/pip/wheels/21/bc/02/74dbd97700a79d4d5d8502f0fc7a92375dabf93e468e2a5a1e
Successfully built emoticon-fix


In [4]:
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.1 MB/s eta 0:00:00


#Import libraries

In [5]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import emoji
from emoticon_fix import emoticon_fix
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

In [6]:
df = pd.read_excel("testing_preprocesing.xlsx")
df.head(11)

,Text
0,@sprintcare and how do you propose we do that
1,@sprintcare I did.
2,game is on 🔥🔥
3,Hilarious😂
4,I am sad :(
5,Driverless AI NLP blog post on https://www.h2o...
6,"<a href=""https://www.h2o.ai/products/h2o-drive..."
7,one minute BRB
8,imo this is awesome
9,thnks for readin the notebook


#Time for some Cleaning!

Before we move ahead , let us clean the dataset and remove the redundancies.

#Abbreviated words in Text

In [7]:
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
AI=Artificial intelligence
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
NLP=Natural language processing
OIC=Oh I See
pls=please
plz=please'
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=thanks
thnx=thanks
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher

"""

#Decoding Function for abbreviations 

In [8]:
chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def abbr_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)


#Replace Emoticon/Emoji with Text , Decoding of abbreviations , 
#Removal of all the hashtags, HTML tags, mentions, punctuations, numbers, and URLs.

In [9]:
def clean_data(text, lang):
    # remove HTML
    soup = BeautifulSoup(text, 'lxml')
    text = soup.get_text()
    #Replace Emoticon/Emoji with Text
    text = emoji.demojize(text, language = lang )
    text = emoticon_fix.emoticon_fix(text)
    #6Decoding of abbreviations 
    text = abbr_conversion(text)
    # remove mentions
    text = re.sub("@[A-Za-z0-9]+","", text)
    # remove hashtags
    text = re.sub("#[A-Za-z0-9_]+","", text)
    # remove links
    text = re.sub('https:\/\/\S+', '', text) 
    # remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text) 
    # remove next line     
    text = re.sub(r'[^ \w\.]', '', text) 
    # remove words containing numbers
    text = re.sub('\w*\d\w*', '', text)
    return text

In [10]:
df['text_Prep'] = df.Text.apply(lambda x: clean_data(x, 'en'))
df.head(10)

<ipython-input-9-21a938c6f932>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')


,Text,text_Prep
0,@sprintcare and how do you propose we do that,and how do you propose we do that
1,@sprintcare I did.,I did
2,game is on 🔥🔥,game is on fire fire
3,Hilarious😂,Hilarious face with tears of joy
4,I am sad :(,I am sad Sad
5,Driverless AI NLP blog post on https://www.h2o...,Driverless Artificial intelligence Natural lan...
6,"<a href=""https://www.h2o.ai/products/h2o-drive...",Driverless Artificial intelligence
7,one minute BRB,one minute Be Right Back
8,imo this is awesome,In My Opinion this is awesome
9,thnks for readin the notebook,thnks for readin the notebook


#Lower Casing

In [11]:
df["text_Prep"] = df["text_Prep"].str.lower()
df.head(10)

,Text,text_Prep
0,@sprintcare and how do you propose we do that,and how do you propose we do that
1,@sprintcare I did.,i did
2,game is on 🔥🔥,game is on fire fire
3,Hilarious😂,hilarious face with tears of joy
4,I am sad :(,i am sad sad
5,Driverless AI NLP blog post on https://www.h2o...,driverless artificial intelligence natural lan...
6,"<a href=""https://www.h2o.ai/products/h2o-drive...",driverless artificial intelligence
7,one minute BRB,one minute be right back
8,imo this is awesome,in my opinion this is awesome
9,thnks for readin the notebook,thnks for readin the notebook


#Fix Misspelled Words 

In [12]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)


In [13]:
df['text_Prep'] = df.text_Prep.apply(lambda x: correct_spellings(x))
df.head(10)

,Text,text_Prep
0,@sprintcare and how do you propose we do that,and how do you propose we do that
1,@sprintcare I did.,i did
2,game is on 🔥🔥,game is on fire fire
3,Hilarious😂,hilarious face with tears of joy
4,I am sad :(,i am sad sad
5,Driverless AI NLP blog post on https://www.h2o...,driverless artificial intelligence natural lan...
6,"<a href=""https://www.h2o.ai/products/h2o-drive...",driverless artificial intelligence
7,one minute BRB,one minute be right back
8,imo this is awesome,in my opinion this is awesome
9,thnks for readin the notebook,thanks for reading the notebook


#Lemmatization

In [18]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["text_Prep"] = df["text_Prep"].apply(lambda text: lemmatize_words(text))
df.head(10)

,Text,text_Prep
0,@sprintcare and how do you propose we do that,and how do you propose we do that
1,@sprintcare I did.,i did
2,game is on 🔥🔥,game is on fire fire
3,Hilarious😂,hilarious face with tear of joy
4,I am sad :(,i am sad sad
5,Driverless AI NLP blog post on https://www.h2o...,driverless artificial intelligence natural lan...
6,"<a href=""https://www.h2o.ai/products/h2o-drive...",driverless artificial intelligence
7,one minute BRB,one minute be right back
8,imo this is awesome,in my opinion this is awesome
9,thnks for readin the notebook,thanks for reading the notebook


In [21]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_Prep"] = df["text_Prep"].apply(lambda text: remove_stopwords(text))
df.head()

,Text,text_Prep
0,@sprintcare and how do you propose we do that,propose
1,@sprintcare I did.,
2,game is on 🔥🔥,game fire fire
3,Hilarious😂,hilarious face tear joy
4,I am sad :(,sad sad
